# 토렌트 마그넷 자동 검색기 만들기

* 난이도 : ★★★☆☆☆☆☆☆☆
* 필요라이브러리: requests, BeautifulSoup4, re



* 검색어를 입력하면 구글링을 통해 토렌트 마그넷을 자동으로 검색해주는 프로그램 입니다.
* 구글링을 통해 나오는 결과기 때문에 어떤 검색어를 입력해도 상관없습니다.
* 강좌에서는 민감한 단어를 예제로 사용하지는 않습니다.
* **PC에 토렌트 클라이언트** 프로그램이 미리 설치되어있어야 합니다.


***해당 프로젝트는 파이썬 강좌의 부분으로 공부 목적으로 제작하는 개인 프로그램 입니다. 대한민국은 토렌트 허용국가지만 저작권 보호 자료를 다운로드하는 행위는 바람직하지 않습니다.***

### 구글에서 검색어 + torrent 를 검색합니다.

![image.png](images/torrent_2.jpg)

* 브라우저 주소창을 확인하여 검색어를 어떻게 질의해야 하는지 확인합니다.
* 개발자도구를 참고하여 구글의 검색결과를 어떻게 접근해야하는지 계획을 세웁니다.

In [ ]:
# 크롤링에 필요한 라이브러리
import requests
from bs4 import BeautifulSoup

# 검색어
keyword = "칼리리눅스"

# 검색어 뒤에 무조건 torrent 를 붙입니다.
keyword += "+torrent"

# 구글 검색 URL 변수 설정
url = "https://www.google.co.kr/search?q={0}&oq={0}".format(keyword)
r = requests.get(url)
bs = BeautifulSoup(r.text, "lxml")

# 검색결과 Selector
divs = bs.select("div.g")
for d in divs:
    print(d)

### 검색된 결과를 파싱 해보면 실제 브라우저에서 보이는것과 다름을 확인 할 수 있습니다.

* 구글은 접속된 브라우저 정보를 확인하여 다른 결과를 주는것으로 추측됩니다.
* 따라서 실제 브라우저처럼 User-Agent 값을 헤더에 붙여서 다시 요청해봅니다.

In [ ]:
# 크롤링에 필요한 라이브러리
import requests
from bs4 import BeautifulSoup

# 검색어
keyword = "칼리리눅스"
# 검색어 뒤에 무조건 torrent 를 붙입니다.
keyword += "+torrent"

# 구글 검색 URL 변수 설정
url = "https://www.google.co.kr/search?q={0}&oq={0}".format(keyword)
r = requests.get(url, headers={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36"})
bs = BeautifulSoup(r.text, "lxml")

# 개발자도구를 참고하여 검색결과의 링크 주소까지 선택하는 선택자를 찾을 수 있습니다.
links = bs.select("div.g > div > div.rc > div.r > a")
for a in links:
    # a 링크에 href 속성이 없으면 계속 반복
    if not a.has_attr('href'):
        continue
        
    # href 속성이 있다면
    href = a["href"]
    print(href)
    

### 검색 결과를 구했으면 해당 결과의 링크로 접속을 해서 마그젯 주소를 찾는다.

* 마그넷 주소의 프로토콜은 magnet:?xt= 로 시작합니다.

In [ ]:
# 크롤링에 필요한 라이브러리
import requests
from bs4 import BeautifulSoup
import re

# 검색어
keyword = "칼리리눅스"
# 검색어 뒤에 무조건 torrent 를 붙입니다.
keyword += "+torrent"

# 구글 검색 URL 변수 설정
url = "https://www.google.co.kr/search?q={0}&oq={0}".format(keyword)
r = requests.get(url, headers={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36"})
bs = BeautifulSoup(r.text, "lxml")

# 개발자도구를 참고하여 검색결과의 링크 주소까지 선택하는 선택자를 찾을 수 있습니다.
links = bs.select("div.g > div > div.rc > div.r > a")
for a in links:
    # a 링크에 href 속성이 없으면 계속 반복
    if not a.has_attr('href'):
        continue
        
    try:
        # href 속성이 있다면 해당 페이지를 접속
        href = a["href"]
        text = a.select("h3")
        title = text[0].text

        r = requests.get(href)
        bs = BeautifulSoup(r.text, "lxml")
        magnets = bs.find_all("a", href=re.compile(r'magnet:\?xt=*'))
        
        if len(magnets) > 0:
            print((title, magnets))
    except:
        continue
    
    

### 페이징 기능 추가
* 기능을 함수화 시키고 페이징 기능을 추가합니다.

In [4]:
# 크롤링에 필요한 라이브러리
import requests
from bs4 import BeautifulSoup
import re # 정규식 사용을 위한 라이브러리


def search_google(keyword, start_page, end_page=None):
    '''구글에서 검색어를 검색해 마그넷 주소를 파싱하는 함수
    Args:
        keyword (str) : 검색어
    
    Returns:
        list : 마그넷주소가 있는 결과 리스트(제목, 마그넷주소)
    '''
    
    # 최종 결과를 리턴할 리스트 변수
    results = []

    # 검색어 뒤에 torrent 가 없으면 붙입니다.
    if keyword.find("+torrent") < 0:
        keyword += "+torrent"

    # 구글 검색 URL 변수 설정
    url = "https://www.google.co.kr/search?q={0}&oq={0}&start={1}".format(keyword, start_page)
    r = requests.get(url, headers={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36"})
    bs = BeautifulSoup(r.text, "lxml")
    
    # 화면에 안내 출력
    print(url, "분석중....")
    
    # end_page 가 None 이면 첫번째 페이지라고 가정
    if end_page is None:
        # 검색결과 Selector
        counts = bs.select("div#resultStats")[0].text
        
        # 검색결과 약 779,000개 (0.24초) 
        # 에서 779000 을 남기고 모두 제거
        counts = counts.replace("검색결과", "").replace("약", "").replace("개", "").replace(",", "").split("(")[0].strip()
        
        # 구글은 페이지당 10개씩 출력
        end_page = int(counts) / 10
        
        # 2 페이지 까지만 검색하게 함
        if end_page > 20:
            end_page = 20

    # 개발자도구를 참고하여 검색결과의 링크 주소까지 선택하는 선택자를 찾을 수 있습니다.
    links = bs.select("div.g > div > div.rc > div.r > a")
    for a in links:
        # a 링크에 href 속성이 없으면 계속 반복
        if not a.has_attr('href'):
            continue

        try:
            # href 속성이 있다면 해당 페이지를 접속
            href = a["href"]
            text = a.select("h3")
            title = text[0].text

            # 링크 주소를 다시 접속
            r = requests.get(href)
            bs = BeautifulSoup(r.text, "lxml")
            
            # 페이지에서 magnet:?xt= 로 시작하는 모든 링크를 추출
            magnets = bs.find_all("a", href=re.compile(r'magnet:\?xt=*'))

            # magnet:?xt= 로 추출된 갯수가 0보다 크면
            if len(magnets) > 0:
                # A 태그에서 실제 링크주소(href) 를 추출
                magnet = magnets[0]["href"]
                # 최종 결과 리스트에 추가
                results.append((title, magnet))
        except:
            continue

    # 현재 페이지가 end_page 보다 작으면 페이징
    if start_page < end_page:
        start_page += 10
        # 재귀함수 호출하고 결과를 results 에 extend 시킴
        results.extend(search_google(keyword, start_page, end_page))
    
    # 최종 결과 리턴
    return results


# 검색어
keyword = "칼리리눅스"
results = search_google(keyword, 0)

for r in results:
    print(r)

https://www.google.co.kr/search?q=칼리리눅스+torrent&oq=칼리리눅스+torrent&start=0  분석중....
https://www.google.co.kr/search?q=칼리리눅스+torrent&oq=칼리리눅스+torrent&start=10  분석중....
https://www.google.co.kr/search?q=칼리리눅스+torrent&oq=칼리리눅스+torrent&start=20  분석중....
('보안 해킹 전문 칼리 리눅스 입니다..torrent - 토렌트왈', 'magnet:?xt=urn:btih:998A024FD134ADD57C345CAEBC76187B116EACF2')
('보안 해킹 전문 칼리 리눅스 입니다..torrent - 토렌트 유틸 - 토렌트맵', 'magnet:?xt=urn:btih:998A024FD134ADD57C345CAEBC76187B116EACF2')
('kali-linux-2018.1-amd64 at Linuxtracker', 'magnet:?xt=urn:btih:2SPAUGIANJIX27VN4XMKZE2VNHTMMZ3S')
